# Demo-notebook for bruk av funksjoner til å regne ut energiavgifter tilknyttet fossilt brensel

*Forfatter: Benedikt Goodman*

I denne teksten vil kun essensen av hva hvert objekt og metode gjør beskrives. Dersom detaljert informasjon om et objekt trenges kan man skrive **help(objektnavn.metodenavn)** eller **help(funksjonsnavn)** for å få tak i informasjonen om hva objektene/funksjonene gjør og hva de godtar av input.

In [ ]:
# Import main libraries
import pandas as pd
import numpy as np
import os

# Change directory until find project root
notebook_path = os.getcwd()
while "pyproject.toml" not in os.listdir():
    os.chdir("../")

# Import math modules
from src.functions.main_program_functions import divider, multiplier, proportion_func, make_sum_column, diff_maker
# Import data utils
from src.functions.main_program_functions import merge_func, subsetter, multi_subsetter, filename_removal_func, associate_codes, rounding_error_dealer, column_tidy_func
# Import verifier functions
from src.functions.main_program_functions import data_integrity_checker, unntak_filler

# Import helperfunction for making mineral oils
from src.functions.main_program_functions import calculate_mineral_oil

# Import data import assets
from src.functions.import_class import DataImporter
from src.functions.metadata_generator import FolderSearcher

# Import export helpers
from src.functions.export_helpers import batch_exporter

# Import testfuncs
from tests.main_program_functions_testing import missing_nrnaaring_tester

# Reset current working directory after local imports
os.chdir(notebook_path)

## Om objektene som er laget for dette produksjonsløpet

### Identifikasjon av data ved hjelp av FolderSearcher

#### Instansiering av objektet
Brukeren navigerer til filstien (path) hvor dataene for beregningsopplegget ligger. Deretter definerer hen en liste med datasett (dataset_list) som skal importereres. Denne listen brukes av FolderSearcher som leter etter mapper på lokasjonen til path som matcher listen. Brukeren kan også anngi datatypen objektet skal lete etter (datatype).

FolderSearcher spør også om start og sluttperiode for import av data. For at denne funksjonen skal fungere må folderne den importerer data fra har år som navn. Objektet tar ikke hensyn til om det er år som mangler i filhirearkiet.

#### Om metodene til FolderSearcher
- *.generate_metadata()* lager FolderSearcher en dataframe filstier, filnavn og hvilken i hvilken mappe filene finnes 
- *.subset_datasets()* beholder kun verdiene (keep_vals=[]) i angitt gruppe (groups=[]). I dette tilfellet fjerner den alle forekomster av energiregnskapet unntatt 2021 fra metadataene. Disse må være lagret i lister, ellers gir objektet feilemelding. Dersom filtreringen ikke inntreffer kan det være fordi keep_vals argumentet er av feil datatype. Filtreringen godtar nemlig strings, ints, floats og det meste annet.
- *.output_df()* spytter ut dataframen objektet har laget.

Ved å dele opp innlesningen av filinformasjon så kan brukeren selv velge hvilke filer som skal importeres da alle pandas sine funksjoner fungerer på dataframen man får ut. Dermed kan man velge ut subsets av data man vil importere dersom man ser at FileSearcher har vært for generøs med hva den har funnet.

In [ ]:
## Definer filsti hvor subfoldere med data finnes
path = '/ssb/stamme02/nasjregn' + '/miljoregnskaper/skatter/arkiv/'

# Skriv hvilke datasett du vil at objektet skal lete etter
datasets = [
    'innbetalte_avgifter',
    'kvotepliktige',
    'omkoding_nr',
    'satser',
    'unntakskatalog',
    'energiregnskapet']

# FolderSearcher lager en dataframe basert på input og hva den funner.
# I denne dataframen finnes alt som trengs av DataImporter for å importere data
# og kategorisere dem. Dette er en helt normal dataframe så brukeren kan endre
# på og filtrere som hen har lyst til*
metadata_df = (FolderSearcher(path, datatype='sas7bdat', dataset_list=datasets)
               .generate_metadata()
               .subset_energiregnskapet()
               .output_df()
              )

# Vis dataframe
metadata_df.head(15)

Alle tilgjengelige metoder har medhørende dokumentasjon som er lett tilgjengelig via pythons help() funksjonalitet.

In [ ]:
# Hvordan vise dokumentasjonen til en metode i et objekt
#help(FolderSearcher)

### Import av data ved hjelp av DataImporter

DataImporter leser inn dataframen med metadata og bruker så denne til å lese inn og kategorisere dataene etter brukerangitte kategorier. Dataene lagres i en **dictionary**. Den bruker kun metadata_df som input argument når den instansieres. Objektet tar inn en hvilken som helst dataframe, men metodene som tilhører objektet antar at 3 kolonner eksisterer i dataframe. Dette er kolonner med:
- komplett filsti til datafilene som skal importeres
- fullt filnavn
- hvilken mappe filstiene ligger i

Objektet antar foreløpig implisitt at dataene den leter i ligger i undermapper.

#### Om metodene til DataImporter
- simple_import() importerer et sett med filer basert på en kolonne med filstier i en dataframe og returnerer en dictionary hvor datasettene ligger sortert per mappe de leses inn fra. Metoden har en mengde input argumenter som gjør den fleksibel i bruk. På det minste trenger den kun hvilken datatype den skal lese inn (filetype). For øvrige input argumenter skriv help(DataImporter.simple_import).
- add_years() legger til mappenavnet datasettet finnes til som variabel i datasettet. Dersom navnet er et år vil datasettene nå inneholde hvilken årgang de er fra.
- categorise_data() sorterer de importerte dataene basert på mappetilhørlighet, filnavn og en liste med brukerangitte kategorier. For å garantere at hver fil havner i riktig kategori er beste å bruke samme liste man brukte for å søke etter data.
    - Dersom man har mange årganger av samme datasett lagret i mange ulike subfoldere så vil **denne reorganiseringen gjøre det mulig å beregne for alle år av gangen da metoden limer alle datasett av samme kategori til en dataframe**. Hoveddtanken bak dette er å gjøre tilbakeregninger enkelt.
- year_duplicate_tidy_func() ser etter duplikatverdier i to kolonner og sletter kolonne nummer to dersom nummer en allerede finnes i datasettet. Dette er for å rette opp i der hvor add_years() er for naiv i å legge til år (f.eks for et datasett fra en folder med årsnavn men som inneholder flere årganger)
- write_data() skriver ut enten den opprinnelige dictionarien importert fra simple_import() (dersom man er usikker på om andre metoder har tullet til dataene kan dette f.eks være bra for debugging) eller den reorganiserte dictionarien laget av categorise_data()

In [ ]:
# Laste inn data og få det på riktig form
data = (DataImporter(metadata_df)
        .simple_import(filetype='sas7bdat') 
        .add_years()
        .categorise_data(dataset_list=datasets)
        .year_duplicate_tidy_func(dupl_col1='aar', dupl_col2='aar_added')
        .sort_df(sort_by=['aar', 'produktkode'])
        .write_data(output_object='sorted_df'))

In [ ]:
data['energiregnskapet'] = calculate_mineral_oil(data['energiregnskapet'])

data['energiregnskapet']

## Eksempel på bruk av funksjonene i main_program_functions.py for å lage produksjonsløpet

In [ ]:
# Ta ut aktuelle subset av kildedata
jet_forb = subsetter(data, key='energiregnskapet', var='produktkode',
                     value='EP04661')

jet_unntak = subsetter(data, key='unntakskatalog', var='produktkode',
                       value='EP04661')

avgifter_jetparafin = subsetter(data, key='innbetalte_avgifter',
                                var='produktkode', value='EP04661')

In [ ]:
avgifter_jetparafin

In [ ]:
# Test of multi_subsetter
multi_subsetter(data, key='innbetalte_avgifter', ytart='41364', produktkode='EP030')

Pandas har en funksjon som tillater at man kan lage en modulær pipeline slik som man gjør i R med %>% operatoren. Dette gjør at man kan bygge enkeltfunksjoner som gjør en del av jobben, og så kan man koble dem sammen slik at de til sammen kjører hele opplegget for en gitt energivare. Dette gjør at man ender opp med et modulært system hvor en selv kan sette rekkefølgen på funksjonene man bruker og hvor man selv står fritt til å definere input-argumenter for hvert trinn.

Denne metoden heter .pipe()

Kort sagt så tillater den at man kan bruke funksjoner man har brukt på egen hånd på dataframes og den lar seg kjede som alle andre pandas metoder.

### Eksempel på hvordan funksjonene kan settes sammen

In [ ]:
jetparafin = (
    # Joins unntaks katalog onto forbruk by aar and næringskode
    merge_func(jet_forb, jet_unntak,
               subset_columns_r=['aar', 'naaringskode', 'unntak', 'ytart'],  # This argument selects columns 
               join_on=['aar', 'naaringskode'],
               join_method='left',
              )
    # removes filename from dataframe column
    #.pipe(filename_removal_func)
    
    # Fills in unntakskatalog
    .pipe(unntak_filler)
    
    # Calculates avgiftsbelagt mengde
    .pipe(multiplier, X='mengde', Y='unntak', new_col='avgiftsbelagt_mengde')
    
    # Calculates total avgiftsbelagt mengde
    .pipe(make_sum_column,
          group='aar',
          target_col='avgiftsbelagt_mengde',
          new_col='total_avgiftsbelagt_mengde')
    
    # Merges avgifter for jetparafin onto main dataframe
    .pipe(merge_func, avgifter_jetparafin,
          subset_columns_r=['aar', 'produktkode', 'total_avgift_kroner'],
          join_on=['aar', 'produktkode'])
    
    # Calculates avgiftsbelagt mengde
    .pipe(proportion_func,
          X='avgiftsbelagt_mengde',
          Y='total_avgiftsbelagt_mengde',
          Z='total_avgift_kroner',
          new_col='est_avgift_kroner')
    
    # Attach NR codes from omkoding to main dataframe
    .pipe(associate_codes,
         df_codes=data['omkoding_nr'],
         ind_code_col='naaringskode',
         nr_code_col = 'nr_naaring',
         prod_name_col = 'produkt_tekst',
         prod_name = 'Jetparafin')
    
    # Checks and attributes diff column to est_avgifter_kroner
    .pipe(rounding_error_dealer)
    
    # Checks for missing nr_codes where est_avgifter_kroner != 0
    # Shows rows of the df if codes are missing
    .pipe(missing_nrnaaring_tester)

    # Reshapes an aggregates main df to nr-code level
    .pipe(column_tidy_func,
        keep_cols=['ytart', 'produkt_tekst', 'aar', 'nr_naaring'],
        avgift_col='est_avgift_kroner')
)

Når alle datasettettene er kommet på formen over kan de limes sammen vi pd.concat og deretter exporteres med batch_exporter funksjonen.

Denne funksjonen oppretter mapper basert på hvert enkelt år i datasettet og eksporterer data fra samme år i samme mappe som ett enkelt datasett.

In [ ]:
# output_df = pd.concat([jetparafin, autodiesel etc...])

In [ ]:
help(batch_exporter)

In [ ]:
#Exports datsets into subfolders
batch_exporter(jetparafin, year_col='aar', 
               export_path='../output_data', # can also be path to anywhere else on linux
               prefix='jet')